In [18]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np

path=r'D:\july_1st_mandrill_export_activity\activity.csv'
latest_file_df = pd.read_csv(path)
latest_file_df['just_date'] = pd.to_datetime(latest_file_df['Date']).dt.normalize()
#latest_file_df.set_index('Date', inplace=True )

print("This is the latest file exported on 1st July 2021")
#print("This is the file exported on 23rd March 2021")
print(latest_file_df.shape)
#print(latest_file_df)
# print("--------------------------------")
# old_file_df = pd.read_csv(r'D:\exports_for_just_activity\activity.csv', )
# old_file_df['just_date'] = pd.to_datetime(old_file_df['Date'])
# old_file_df.set_index('Date', inplace=True )            
# print(old_file_df.shape)
#print(old_file_df)

#dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

# path =r'D:\Mandrill_Exports\june_mandrill'
# filenames = glob.glob(path + "/*.csv")

latest_file_df['just_date'] =  pd.to_datetime(latest_file_df['Date'])
#latest_file_df['Date'] = latest_file_df['Date'].datetime.date
latest_file_df['just_date'] = latest_file_df['just_date'].dt.date

#print(latest_file_df['Date'])
#print(latest_file_df['just_date'])

max_date = latest_file_df["just_date"].max()
latest_date = max_date.date()
min_date = latest_file_df["just_date"].min()
oldest_date = min_date.date()
print("Latest date for exports : ", latest_date)
print("Oldest date for exports : ", oldest_date)
date_range = []
date_range.append(oldest_date)

while oldest_date < latest_date :
    oldest_date = oldest_date + timedelta(days=1)
    date_range.append(oldest_date)

for dates in date_range:
    daily_wise_file_df = latest_file_df.loc[latest_file_df['just_date'] == dates]
    path1 = r'D:\Automated_mandrill_files\daily_wise_file_from_23_03_2021\date_wise_files'
    file_name = path1 + "mandrill_data" + str(dates) + ".csv"
    daily_wise_file_df.to_csv(file_name, index=False)

    
    
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(filename, parse_dates=['just_date'], date_parser=dateparse))
# big_frame = pd.concat(dfs, ignore_index=True)

This is the latest file exported on 1st July 2021
(61968, 11)
Latest date for exports on 21 June 2021 :  2021-07-01
Oldest date for exports on 21 June 2021 :  2021-04-01


In [30]:
import glob

dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

path1 =r'D:\Automated_mandrill_files\daily_wise_file_from_23_03_2021'
filenames = glob.glob(path1 + "/*.csv")

dfs = []
counts_original_df = []
counts_downsizing = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, parse_dates=['just_date'], date_parser=dateparse))
big_frame = pd.concat(dfs, ignore_index=True)
print(big_frame.shape)

max_date = big_frame["just_date"].max()
latest_date = max_date.date()
min_date = big_frame["just_date"].min()
oldest_date = min_date.date()
print("-------------------------------")
print("Latest date : ", latest_date)
print("Oldest date : ", oldest_date)
print("-------------------------------")
total_recipients = 0
visited = []
for i in range(0, len(big_frame['Email Address'])):
    if big_frame['Email Address'][i] not in visited: 
        visited.append(big_frame['Email Address'][i])
        total_recipients += 1
print("No.of.unique Email Address values :", total_recipients)
print("--------------------------------------------")
print(big_frame.shape)
print("-------before removing bogus emails---------")
print()

def remove_bogus_emails(df):
    """
    Removes test emails & special emails such as the ones used by developer partners
    """
    unwanted_emails_list =  ['testing1ajeet@gmail.com']
    #df = df.loc[df['Email'].str.contains("@")==True]
    df = df.loc[df['Email Address'].str.contains("yopmail")==False]
    df = df.loc[df['Email Address'].str.contains("ypomail")==False]
    df = df.loc[df['Email Address'].str.contains("gopmail")==False]
    df = df.loc[df['Email Address'].str.contains("skidos.com")==False]
    df = df.loc[~df['Email Address'].isin(unwanted_emails_list)]
    return df

big_frame = remove_bogus_emails(big_frame)
print(big_frame.shape)
print("---------after removing bogus emails---------")

def remove_status_other_sent(df):
    """
    Removes rows with Status other than sent
    """
    status_strings =  ['soft-bounced',
                       'hard-bounced',
                       'spam','rejected',
                       'defered', 'bounced']
    df = df.loc[~df['Status'].isin(status_strings)]
    return df

big_frame = remove_status_other_sent(big_frame)
print(big_frame.shape)
print("---------after removing rows with status other than sent---------")


agg_obj_1 = big_frame.groupby(['just_date', 'Subject'],as_index=False)
agg_df_1 = pd.DataFrame(agg_obj_1.agg({'Email Address' : 'nunique'}))
agg_df_1.rename(columns={'Email Address':'unique_recipients'}, inplace=True)
print(agg_df_1.shape)
print(agg_df_1.head())
print()

agg_obj_2 = big_frame.groupby(['just_date', 'Subject'],as_index=False)
agg_df_2 = pd.DataFrame(agg_obj_2.agg({'Email Address' : 'count'}))
agg_df_2.rename(columns={'Email Address':'total_recipients'}, inplace=True)
print(agg_df_2.shape)
print(agg_df_2.head())

max_date = agg_df_1["just_date"].max()
latest_date = max_date.date()
min_date = agg_df_1["just_date"].min()
oldest_date = min_date.date()
print("-------------------------------")
print("Latest date : ", latest_date)
print("Oldest date : ", oldest_date)

# final_df = pd.merge(agg_df_1, agg_df_2, left_on=['just_date', 'Subject'], right_on=['just_date', 'Subject'], how='inner')
# final_df.to_csv(r'D:\Data\New folder\mandril_activity_anlysis\mandrill_emails_counts_with_subjects.csv', index=False)
# print("File saved")

(63120, 11)
-------------------------------
Latest date :  2021-07-01
Oldest date :  2021-03-23
-------------------------------
No.of.unique Email Address values : 53458
--------------------------------------------
(63120, 11)
-------before removing bogus emails---------

(59197, 11)
---------after removing bogus emails---------
(51455, 11)
---------after removing rows with status other than sent---------
(651, 3)
   just_date                   Subject  unique_recipients
0 2021-06-01      Bem-vindo ao Skidos!                 56
1 2021-06-01        Benvenuto a skidos                 15
2 2021-06-01        Bienvenue à skidos                 23
3 2021-06-01  Forgotten your password?                 16
4 2021-06-01            Glemt passord?                  1

(651, 3)
   just_date                   Subject  total_recipients
0 2021-06-01      Bem-vindo ao Skidos!                56
1 2021-06-01        Benvenuto a skidos                15
2 2021-06-01        Bienvenue à skidos               

In [29]:
total_recipients = 0
visited = []
for i in range(0, len(big_frame['Email Address'])):
    if big_frame['Email Address'][i] not in visited: 
        visited.append(big_frame['Email Address'][i])
        total_recipients += 1
print("No.of.unique Email Address values :", total_recipients)
print("--------------------------------------------")
print(big_frame.shape)
print("-------before removing bogus emails---------")
print()

def remove_bogus_emails(df):
    """
    Removes test emails & special emails such as the ones used by developer partners
    """
    unwanted_emails_list =  ['testing1ajeet@gmail.com']
    #df = df.loc[df['Email'].str.contains("@")==True]
    df = df.loc[df['Email Address'].str.contains("yopmail")==False]
    df = df.loc[df['Email Address'].str.contains("ypomail")==False]
    df = df.loc[df['Email Address'].str.contains("gopmail")==False]
    df = df.loc[df['Email Address'].str.contains("skidos.com")==False]
    df = df.loc[~df['Email Address'].isin(unwanted_emails_list)]
    return df

big_frame = remove_bogus_emails(big_frame)
print(big_frame.shape)
print("---------after removing bogus emails---------")

def remove_status_other_sent(df):
    """
    Removes rows with Status other than sent
    """
    status_strings =  ['soft-bounced',
                       'hard-bounced',
                       'spam','rejected',
                       'defered', 'bounced']
    df = df.loc[~df['Status'].isin(status_strings)]
    return df

big_frame = remove_status_other_sent(big_frame)
print(big_frame.shape)
print("---------after removing rows with status other than sent---------")


agg_obj_1 = big_frame.groupby(['just_date', 'Subject'],as_index=False)
agg_df_1 = pd.DataFrame(agg_obj_1.agg({'Email Address' : 'nunique'}))
agg_df_1.rename(columns={'Email Address':'unique_recipients'}, inplace=True)
print(agg_df_1.shape)
print(agg_df_1.head())
print()

agg_obj_2 = big_frame.groupby(['just_date', 'Subject'],as_index=False)
agg_df_2 = pd.DataFrame(agg_obj_2.agg({'Email Address' : 'count'}))
agg_df_2.rename(columns={'Email Address':'total_recipients'}, inplace=True)
print(agg_df_2.shape)
print(agg_df_2.head())

max_date = agg_df_1["just_date"].max()
latest_date = max_date.date()
min_date = agg_df_1["just_date"].min()
oldest_date = min_date.date()
print("-------------------------------")
print("Latest date : ", latest_date)
print("Oldest date : ", oldest_date)

# final_df = pd.merge(agg_df_1, agg_df_2, left_on=['just_date', 'Subject'], right_on=['just_date', 'Subject'], how='inner')
# final_df.to_csv(r'D:\Data\New folder\mandril_activity_anlysis\mandrill_emails_counts_with_subjects.csv', index=False)
# print("File saved")

KeyError: 0

In [11]:
print (latest_file_df.index.min())
print (latest_file_df.index.max())
print("--------------------------------")
print (old_file_df.index.min())
print (old_file_df.index.max())

2021-04-01 09:32:46
2021-07-01 09:30:11
--------------------------------
2021-03-26 00:05:11
2021-06-24 11:17:31


In [8]:
min(latest_file_df['just_date'])
max(latest_file_df['just_date'])
print("--------------------------------")
min(old_file_df['just_date'])
max(old_file_df['just_date'])

--------------------------------


Timestamp('2021-06-24 11:17:31')